# Introduction to python for hydrologists &mdash; pandas

## Exploring historical time series from Weather Underground

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import zipfile
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.backends.backend_pdf import PdfPages

# Weather Underground Serves Local Weather data from Airports -- like SFO
## We can pull it using historical data API, although currently offline....      :(

## But....we can read it in from an Excel file we saved earlier

In [ ]:
sfo_weather = pd.read_excel(os.path.join('data','pandas','wug_SFO.xlsx'),sheet_name='weather_data',
                          index_col=[0])

## We are going to need to convert the time index - formats described at:

https://docs.python.org/3/library/datetime.html

In [ ]:
sfo_weather.index = pd.to_datetime(sfo_weather.index, 
                                   format='%Y-%m-%d %H:%M:%S')

In [ ]:
sfo_weather.index[0]

In [ ]:
sfo_weather.resample('M').max().PrecipitationIn.plot()

## We can aggregate over various times and calculate statistics

In [ ]:
g = sfo_weather.groupby(sfo_weather.index.year)

In [ ]:
g.aggregate([np.min, np.mean, np.max])['PrecipitationIn'].plot()

In [ ]:
sfo_weather.columns

In [ ]:
sfo_weather['Wind Direction'].unique()

## let's convert these to degrees and radians and make a rose plot

Conversions:
http://blog.weatherflow.com/degrees-of-wind-direction-along-south-carolina-onshore-vs-offshore/

In [ ]:
dir_str = ['WNW', 'Calm', 'NNW', 'NNE', 'North', 'NW', 'South', 'West', 'SSW',
       'Variable', 'WSW', 'SW', 'NE', 'SSE', 'SE', 'East', 'ENE', 'ESE']
dir_deg = np.array([52, 0, 60, 20, 64, 56, 32, 48, 36,
               0, 44, 40, 8, 28, 24, 16, 12, 20])*360/64

In [ ]:
wind_lookup = dict(zip(dir_str,dir_deg))

In [ ]:
sfo_weather['wind_deg'] = [wind_lookup[i] for i in sfo_weather['Wind Direction']]

### Convert to Radians

In [ ]:
sfo_weather['wind_rad'] = sfo_weather.wind_deg* np.pi/180

In [ ]:
sfo_weather['wind_deg'].hist(bins=50)

In [ ]:
sfo_weather['wind_rad'].hist(bins=50)

In [ ]:
counts, bins = np.histogram(sfo_weather['wind_rad'], bins=50)

In [ ]:
counts

In [ ]:
bins

In [ ]:
bins=(bins[1:]+bins[:-1])/2

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8], polar=True)
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)
widths = np.pi/4e5*counts
bars = ax.bar(bins,counts, width=widths)


## Cool -- is it seasonal?

In [ ]:
# let's grab the calendar module to convert month numbers to names
import calendar

In [ ]:
with PdfPages(os.path.join('data','monthly_wind_dir.pdf')) as outpdf:
    for cn, cg in sfo_weather.groupby(sfo_weather.index.month):
        print('plotting for {0}'.format(calendar.month_name[cn]))
        counts, bins = np.histogram(cg['wind_rad'], bins=50)
        bins = (bins[1:]+bins[:-1])/2
        fig = plt.figure()
        ax = fig.add_axes([0.1, 0.1, 0.8, 0.8], polar=True)
        ax.set_theta_zero_location("N")
        ax.set_theta_direction(-1)
        widths = np.pi/4e4*counts
        bars = ax.bar(bins,counts, width=widths)
        plt.title(calendar.month_name[cn])
        outpdf.savefig()
        plt.close()

## Look at wind speed --- needs some cleaning

In [ ]:
sfo_weather['Wind SpeedMPH'].unique()

In [ ]:
sfo_weather.dtypes

In [ ]:
sfo_weather['Wind SpeedMPH'] = [0 if i == 'Calm' else float(i) for i in sfo_weather['Wind SpeedMPH']]

In [ ]:
sfo_weather['Wind SpeedMPH'].hist(bins=50)

In [ ]:
len(sfo_weather.loc[sfo_weather['Wind SpeedMPH']>50])

In [ ]:
sfo_weather['Wind SpeedMPH'] = [0 if i == 'Calm' else np.nan if float(i) > 55 else float(i) for i in sfo_weather['Wind SpeedMPH']]

In [ ]:
sfo_weather['Wind SpeedMPH'].describe()

## Is there fog at SFO?

In [ ]:
sfo_weather.VisibilityMPH.unique()

In [ ]:
sfo_weather.VisibilityMPH[sfo_weather.VisibilityMPH>50].count()

In [ ]:
sfo_weather.loc[sfo_weather.VisibilityMPH>50,'VisibilityMPH'] = np.nan

In [ ]:
sfo_weather.VisibilityMPH[sfo_weather.VisibilityMPH>50].count()

In [ ]:
sfo_weather.VisibilityMPH.hist()

In [ ]:
for cn, cg in sfo_weather.groupby(sfo_weather.index.month):
    plt.figure()
    cg.VisibilityMPH.hist()
    plt.title(calendar.month_name[cn])
    

## Let's look at Gusts. More data cleaning!

In [ ]:
sfo_weather['Gust SpeedMPH'].unique()

In [ ]:
sfo_weather.loc[sfo_weather['Gust SpeedMPH']=='-','Gust SpeedMPH'] = np.nan
sfo_weather['Gust SpeedMPH'] = sfo_weather['Gust SpeedMPH'].astype(float)

In [ ]:
sfo_weather['Gust SpeedMPH'].hist()

In [ ]:
sfo_weather['Gust SpeedMPH'].groupby(sfo_weather.index.year).aggregate([np.min, np.mean, np.max]).plot(kind='bar')

In [ ]:
sfo_weather['Gust SpeedMPH'].groupby(sfo_weather.index.month).aggregate([np.min, np.mean, np.max]).plot(kind='bar')

## Which is the most variable month for gusts

In [ ]:
fig = plt.figure(figsize=(10,4))
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
sfo_weather['Gust SpeedMPH'].groupby(sfo_weather.index.month).aggregate(np.std).plot(kind='bar')
ax.set_xticklabels([calendar.month_name[i] for i in list(range(1,13))]);

In [ ]:
sfo_weather['Gust SpeedMPH'].groupby(sfo_weather.index.day).aggregate(
    [np.min, np.mean, np.max]).plot(kind='bar', figsize=(10,4))